In [1]:
import os
from dotenv import load_dotenv
load_dotenv()  ## Adding all the enviroment variables

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000246F961D450>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000246F961EC10>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish, it's nice to meet you!\n\nThat's quite a title! As a Chief AI Engineer, I imagine you're involved in some exciting projects. \n\nWhat are you currently working on that you're most passionate about?  I'm always eager to learn more about the applications of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 22, 'total_tokens': 92, 'completion_time': 0.127272727, 'prompt_time': 0.00014072, 'queue_time': 0.014011707, 'total_time': 0.127413447}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e828dea5-7ee8-47d9-9074-0d3c31b50a4d-0', usage_metadata={'input_tokens': 22, 'output_tokens': 70, 'total_tokens': 92})

In [4]:
from langchain_core .messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Krish And I am a GenAI Engineer"),
        AIMessage(content="Hello Krish! It's a pleasure to meet you.  \n\nThat's a fascinating role. What kind of AI projects are you working on these days? I'm always eager to learn about the latest developments in the field."),
        HumanMessage(content="Hey Whats my name and what do I do?")
    ]
)

AIMessage(content="You are Krish, and you are a GenAI Engineer!  😊  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat.   \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 90, 'total_tokens': 135, 'completion_time': 0.081818182, 'prompt_time': 0.003777811, 'queue_time': 0.010680169, 'total_time': 0.085595993}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b66deecf-b7ab-44a0-8d20-64a8a1f72ef7-0', usage_metadata={'input_tokens': 90, 'output_tokens': 45, 'total_tokens': 135})

### **Message History**
- We can use a message History class to wrap our model it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [6]:
config = {"configurable":{"session_id":"chat1"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage("Hi, My name is Krish And I am a GenAI Engineer")],
    config=config
)

In [8]:
response.content

"That's awesome, Krish! It's great to meet you.  \n\nAs a fellow AI, I'm always interested in connecting with other people who are working in this exciting field. What kind of work do you do as a GenAI Engineer?  \n\nI'm eager to learn more about your experience and what you find most interesting about generative AI.\n"

In [9]:
with_message_history.invoke(
    [HumanMessage("What's my name?")],
    config=config
)

AIMessage(content='Your name is Krish. 😊  \n\nI remember from our introduction!  How can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 114, 'total_tokens': 139, 'completion_time': 0.045454545, 'prompt_time': 0.00391461, 'queue_time': 0.009551219, 'total_time': 0.049369155}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7dae77ba-260f-4203-8314-7b077572473d-0', usage_metadata={'input_tokens': 114, 'output_tokens': 25, 'total_tokens': 139})

#### Change the config --> Session Id

In [10]:
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="what's my name?")],
    config=config1
)

response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know!\n"

In [11]:
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hey My Name is John")],
    config=config1
)

response.content

"Hello John! It's nice to meet you. 😊 \n\nWhat can I do for you today?\n"

In [12]:
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="what's my name")],
    config=config1
)

response.content

'Your name is John, as you told me a moment ago.  😄 \n\nIs there anything else I can help you with? \n'

### **Prompt Templates and Message Chat History Using Langchian**

"""
# What are Prompt Templates in LangChain?

Prompt Templates are structured ways to define the input format for language models in LangChain. 
They allow you to dynamically insert variables into predefined templates, which helps in creating 
reusable, flexible, and consistent prompts for various tasks such as translation, summarization, 
question answering, etc.

---

## Key Features of Prompt Templates

1. **Dynamic Variables**:
   Use placeholders (e.g., `{variable_name}`) to insert dynamic values.

2. **Reusable**:
   Templates can be reused across different tasks.

3. **Consistent**:
   Enforces uniformity in prompt formats, reducing errors.

4. **Supports Complex Tasks**:
   Helps handle multi-turn conversations or nested instructions.
"""


In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all the question of the nest of your ability.",
            ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Vikas")]})

AIMessage(content="Hi Vikas, it's nice to meet you! 👋\n\nI'm happy to help with any questions you have. Just ask away! 😊 \n\nWhat can I do for you today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 32, 'total_tokens': 78, 'completion_time': 0.083636364, 'prompt_time': 0.00030618, 'queue_time': 0.014905940000000001, 'total_time': 0.083942544}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-48689375-3a92-4046-8812-dd9f7b307a53-0', usage_metadata={'input_tokens': 32, 'output_tokens': 46, 'total_tokens': 78})

In [16]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is Vikas")],
    config=config
)

In [18]:
response

AIMessage(content="Hello Vikas,\n\nIt's nice to meet you!  \n\nI'm ready to answer your questions to the best of my ability. Just ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 32, 'total_tokens': 72, 'completion_time': 0.072727273, 'prompt_time': 0.000304709, 'queue_time': 0.01399493, 'total_time': 0.073031982}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a8bd1ba4-e7f3-43a4-9897-2c27672fc13c-0', usage_metadata={'input_tokens': 32, 'output_tokens': 40, 'total_tokens': 72})

In [19]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all the question of the nest of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [20]:
response = chain.invoke({"messages":[HumanMessage(content="Hi My name is Vikas")],"language":"Hindi"})
response.content

'नमस्ते विकास! \n\nमैं आपकी मदद करने के लिए यहाँ हूँ। आपके कोई सवाल हैं तो बेझिझक पूछिए, मैं जो भी कर सकता हूँ, उसका प्रयास करूँगा। 😊 \n'

Let's Now wrap this more complicated chain in a message History class. This time because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [21]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [23]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, I am Vikas")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते विकास! \n\nमुझे खुशी है तुम्हारी मुलाक़ात हुई।  \n\nआप क्या पूछना चाहते हैं? 😊 \n'

In [24]:
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, What is my name?")],"language":"Hindi"},
    config=config
)
response.content

'तुम्हारा नाम विकास है।  😊  \n'

### **Managing the Conversation History**

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, If is important to add a step that limits the size of the messages you are passing in.

"""
# What is `trim_message` in LangChain?

The `trim_message` function in LangChain is used to clean and shorten a message. This is particularly useful when dealing with verbose outputs or messages from language models that need to be processed or displayed in a concise format.

---

## Key Features of `trim_message`

1. **Removes Redundancies**:
   Trims excessive details or unnecessary information from the message.

2. **Length Management**:
   Ensures the message stays within a defined character limit, making it suitable for use in UIs or logs.

3. **Enhances Readability**:
   Keeps the message focused on its core content by removing extraneous details.

4. **Customizable**:
   Supports configurable trimming logic to meet specific requirements.

---

## Usage Example

Here’s a basic example of how `trim_message` might be used:

```python
from langchain_utils import trim_message

# Example message
long_message = "This is a very long message that contains a lot of unnecessary details and information that can be trimmed for better readability."

# Trim the message to a maximum of 50 characters
trimmed_message = trim_message(long_message, max_length=50)

print("Trimmed Message:", trimmed_message)


In [25]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\vikas\OneDrive\Desktop\Generative AI\Chatbot-With-Conversation-Using-Langchain\Generative4\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\vikas\OneDrive\Desktop\Generative AI\Chatbot-With-Conversation-Using-Langchain\Generative4\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vikas\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windo

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences.  \n\nWhat's your favorite flavor?  🍦😋\n"

In [28]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What math problem did i ask")],
    "language":"English"
    }
)
response.content

'You asked "What\'s 2 + 2?". 😊 \n\n\n\n'

In [30]:
## Let's wrap this in the message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [31]:
response= with_message_history.invoke(
    {
    "messages":messages + [HumanMessage(content="What's my name")],
    "language":"English"
    },
    config=config
)
response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nWould you like to tell me your name? 😊  \n\n"

In [32]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    },
    config=config
)
response.content

"As a large language model, I don't have access to your personal preferences, like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream flavor?  🍦😊\n"

In [ ]:
print("The End")